# Analyze histogram of predicted scores

In [ ]:
import sys
sys.path.append("./../")

In [ ]:
import os
import json
import glob

import numpy as np
from tbparse import SummaryReader

import methods, models, datasets, transforms

import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
def get_event_file(model_dir):
    tblog_dir = os.path.join(model_dir, "tblog")
    events_file = glob.glob(tblog_dir + "/events.out.tfevents*.1")[0] # *.1 will be from test

    return events_file

In [ ]:
model_dir = "./../zoo/test/BinaryMNISTC-53-identity/LeNet/20220402190833/"

# Default paths
config_json = os.path.join(model_dir, "config.json")
ckpt_file = os.path.join(model_dir, "last.ckpt")
event_file = get_event_file(model_dir)


In [ ]:
config = json.load(open(config_json, 'r'))
params = config['method_params']

ev = SummaryReader(event_file, pivot=True)
df_scalars = ev.scalars
df_hist = ev.histograms

In [ ]:
df_scalars

In [ ]:
l = df_hist.iloc[-1]
x = l['test_y_pred/limits']
y = l['test_y_pred/counts']
y = y / y.sum()
fig = plt.figure(figsize=(6,4))
# plt.fill_between(x, y, step='pre')
plt.bar(x, y, width=-0.09, align='edge')
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 0.6)
plt.title(
    "Beta({:.2f}, {:.2f})".format(params['a'], params['b']) +
    "    $\\alpha$ = {:.0f}".format(params['alpha']) +
    "    $\\lambda_{SL}$ = "+"{:.2f}".format(params['lam_sl'])
)
plt.xlabel("p")
plt.ylabel("$\hat{s_{obs}}$")